In [1]:
import pandas as pd
import time
import threading
from kafkaconsumer import Kafka_Consumer

In [3]:
def get_env_params():
    return pd.read_csv('../env_params/env_params.csv',index_col=0, header=None).squeeze().to_dict()

In [ ]:
if __name__ == "__main__":
    
    params = get_env_params()

    topics = params['topics'].split('|')


    kafkaconsumers = []
    for iter,topic in enumerate(topics):
        #Create objects of kafka consumer
        kafkaconsumers.append(Kafka_Consumer(topic,[params['bootstrapservers']],str(iter)))
  
    threads = []
    for consumer in kafkaconsumers:
        # Create threads for each consumer function
        thread = threading.Thread(target=consumer.consume_and_upload
                               ,args=(params['adls_account_name'], params['adls_sas_token']
                                      ,int(params['kafkabatchsize']),params['adls_container']))
        threads.append(thread)
        thread.start()

    try:
        while True:
            time.sleep(0.1)  # Main thread does something here, could be anything
    except KeyboardInterrupt:
        print("\nKeyboard interruption detected. Stopping all threads...")
    finally:
        # Wait for all threads to complete

        for consumer in kafkaconsumers:
            consumer.consumer.close()
        
        for thread in threads:
            thread.join()
        print("All threads have been stopped.")